In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors


## Define folders

In [ ]:
dir_data = '/Data/data_ELUC_NGHGI_SLAND_plot/'
dir_ctrs = '/Data/data_ancillary/info_countries/'
dir_fig  = '/Figures/'


## Read data

In [ ]:
#Select time and forest mask year
time_selection   = '2001-2015'
forest_mask_year = '2013'

#Read ELUC and NGHGI
file_name = dir_data + 'Collection_ELUC_NGHGI_' + time_selection + '.pickle'
data_ELUC_NGHGI = pd.read_pickle(file_name)

#Read SLAND in non-intact forests
file_name = dir_data + 'Collection_SLAND_non-intact-forest_mask' + forest_mask_year + '_' + time_selection + '.pickle'
data_SLAND_forest = pd.read_pickle(file_name)

#Read countries
fname_cntrs = dir_ctrs + 'Country codes 3 letters.xlsx'
data_cntrs  = pd.read_excel(fname_cntrs, sheet_name=0, header=None, index_col=0)


## Uncertainty of NGHGIs

In [ ]:
data_unc = pd.DataFrame(columns=['uncertainty_lower', 'uncertainty_upper'])
data_unc.loc['BRA']     = [-34.9, 34.9]
data_unc.loc['CHN']     = [-23.4, 23.4]
data_unc.loc['IDN']     = [-20.1, 20.1]
data_unc.loc['COD']     = [-np.NaN, np.NaN]
data_unc.loc['CAN']     = [-209.0, 209.0]
data_unc.loc['EU27_UK'] = [-34.2, 34.2]
data_unc.loc['RUS']     = [-42.8, 42.8]
data_unc.loc['USA']     = [-17.1, 19.7]
data_unc = data_unc.astype(float)


## Plot preparation

In [ ]:
#Define colors
col_SLAND_for = '#01665e'
col_SLAND_tot = '#80cdc1'
col_ELUC      = 'sienna'

#Fontsize of legend
fs_leg = 15.5

#Select countries
countries_sel = ['USA', 'RUS', 'EU27_UK', 'CAN', 'CHN', 'BRA', 'IDN', 'COD']

#Convert unit from Tg C/year to Pg CO2/year
data_ELUC_NGHGI_sel = 44/12 / 1000 * data_ELUC_NGHGI.loc[countries_sel]
data_SLAND_for_sel  = 44/12 / 1000 * data_SLAND_forest.loc[countries_sel]

#Width of boxplot boxes
wid = 0.125

#Calculate colors for uncertainty
fraction = 0.6
col1 = np.array(colors.to_rgba(col_ELUC)[0:3]) * 255
col2 = np.array([0, 0, 0])
col_ELUC_2 = ( (col1 - col2) * fraction + col2 ) / 255

#Calculate colors for uncertainty
fraction = 0.6
col1 = np.array(colors.to_rgba(col_SLAND_for)[0:3]) * 255
col2 = np.array([0, 0, 0])
col_SLAND_for_2 = ( (col1 - col2) * fraction + col2 ) / 255

#Calculate colors for uncertainty
fraction = 0.6
col1 = np.array(colors.to_rgba(col_SLAND_tot)[0:3]) * 255
col2 = np.array([0, 0, 0])
col_SLAND_tot_2 = ( (col1 - col2) * fraction + col2 ) / 255

#Calculate colors for uncertainty
fraction = 0.6
col1 = np.array(colors.to_rgba('white')[0:3]) * 255
col2 = np.array([0, 0, 0])
col_NGHGI_unc = ( (col1 - col2) * fraction + col2 ) / 255


## Calculate uncertainty for combined estimate of ELUC + SLAND in managed forests

In [ ]:
#Define models
BMs   = ['BLUE', 'HN', 'OSCAR']
DGVMs = [col for col in data_SLAND_forest.columns if  "median" not in col]

#Loop over countries
data_unc_comp = dict()
for country in countries_sel:
    
    #Get SLAND of all DGVMs
    data_SLAND = np.array(data_SLAND_for_sel.loc[country, DGVMs])

    #Loop over BMs
    data_coll = np.array([])
    for BM in BMs:

        #Read BM data
        data_BM = data_ELUC_NGHGI_sel.loc[country, 'ELUC_' + BM + '_2021']

        #Collect all data in array
        data_coll = np.concatenate((data_coll, data_BM + data_SLAND))

    #Calculate IQR and save in dictionary
    data_unc_comp[country] = 1000 * (np.quantile(data_coll, 0.75) - np.quantile(data_coll, 0.25))
    

## Plot NGHGI, ELUC, SLAND

In [ ]:
show_BK_all = 0

include_SLANDtot = 0

#Get column names of DGVMs
models_SLAND = [col for col in data_SLAND_forest.columns if  "median" not in col]

fig, ax = plt.subplots(1, 1, figsize=(18 * len(countries_sel)/10, 5))

gap_before = 0
gap_after = 0

#Loop over all IPCC countries
ctrs = []
ix = 0
hline = True
for i, (country, row) in enumerate(data_ELUC_NGHGI_sel.iterrows()):
    
    #Extract values
    NGHGI           = data_ELUC_NGHGI_sel.loc[country, 'NGHGI']
    NGHGI_unc       = data_unc.loc[country]
    ELUC_BLUE_2021  = data_ELUC_NGHGI_sel.loc[country, 'ELUC_BLUE_2021']
    ELUC_HN_2021    = data_ELUC_NGHGI_sel.loc[country, 'ELUC_HN_2021']
    ELUC_OSCAR_2021 = data_ELUC_NGHGI_sel.loc[country, 'ELUC_OSCAR_2021']
    SLAND_forest    = np.array(data_SLAND_for_sel.loc[country, models_SLAND])
    SLAND_for_med   = data_SLAND_for_sel.loc[country].median()
    
    #Add REDD+ values for COD, and define uncertainties
    if country=='COD':
        NGHGI_REDD = 44/12 / 1000 * data_ELUC_NGHGI.loc['COD_REDD+', 'NGHGI']
        NGHGI_unc = [np.min([NGHGI, NGHGI_REDD]), np.max([NGHGI, NGHGI_REDD])]
        NGHGI = np.mean([NGHGI, NGHGI_REDD])
    else:
        NGHGI_unc = [NGHGI * (1 + NGHGI_unc['uncertainty_lower']/100), NGHGI * (1 + NGHGI_unc['uncertainty_upper']/100)]
    
    #Calculate absolute and relative gaps with all ELUC and SLAND combinations
    ELUCs = np.array([ELUC_BLUE_2021, ELUC_HN_2021, ELUC_OSCAR_2021])

    #Remove NaN values
    ELUCs = ELUCs[~np.isnan(ELUCs)]
    
    #Calculate data for plotting
    ELUC_val      = np.mean(ELUCs)
    SLAND_for_bot = ELUC_val + SLAND_for_med
    SLAND_for_top = -SLAND_for_med
    resid_val_top = NGHGI - (ELUC_val + SLAND_for_med)
    resid_val_bot = ELUC_val + SLAND_for_med
    NGHGI_unc_val = [NGHGI_unc[0], NGHGI_unc[1]]
    
    #Set alpha value to plot SLAND (and SLAND gap) for Canada transparent
    if country=='CAN':  alpha_CAN = 1/3
    else:               alpha_CAN = 1
    
    #Define x positions
    x1 = i + 4/25
    x2 = i + 8/25
    x3 = i + 14/25
    x4 = i + 20/25
    x5 = i + 22/25
    
    #Plot bars    
    p1 = ax.bar(x1, ELUC_val, wid, color=col_ELUC, zorder=30)
    p2 = ax.bar(x2, SLAND_for_top, wid, bottom=SLAND_for_bot, color=col_SLAND_for, alpha=alpha_CAN, zorder=18)
    p4 = ax.bar(x3, NGHGI, wid, color='k', zorder=30)

    if show_BK_all==1:
        thick = 10
        p7 = ax.bar(x1, thick, 1.2 * wid, bottom=ELUCs[0] - thick/2, color='dodgerblue', zorder=200)
        p8 = ax.bar(x1, thick, 1.2 * wid, bottom=ELUCs[1] - thick/2, color='crimson', zorder=200)
        p9 = ax.bar(x1, thick, 1.2 * wid, bottom=ELUCs[2] - thick/2, color='orange', zorder=200)
            
    #Plot uncertainties
    q25_SLAND_for = np.quantile(SLAND_forest, 0.25)
    q75_SLAND_for = np.quantile(SLAND_forest, 0.75)
    ax.bar(x1, np.max(ELUCs) - np.min(ELUCs), wid, bottom=np.min(ELUCs), facecolor='none', edgecolor=col_ELUC_2, hatch="///", linewidth=0, zorder=40)
    ax.bar(x2, q75_SLAND_for - q25_SLAND_for, wid, bottom=ELUC_val + q25_SLAND_for, facecolor='none', edgecolor=col_SLAND_for_2, hatch="///", alpha=alpha_CAN, linewidth=0, zorder=40)
    ax.bar(x3, max(NGHGI_unc_val) - min(NGHGI_unc_val), wid, bottom=min(NGHGI_unc_val), facecolor='none', edgecolor=col_NGHGI_unc, hatch="///", linewidth=0, zorder=40)

    #Plot gap
    p5 = ax.bar(x4, NGHGI-ELUC_val, 0.55 * wid, bottom=ELUC_val, color='indianred', zorder=25)
    p6 = ax.bar(x5, resid_val_top, 0.55 * wid, bottom=resid_val_bot, color='#fec500', alpha=alpha_CAN, zorder=20)

    #Plot lines from one bar to the next one
    ax.plot([x1 - 3/5 * wid, x2 +  3/5 * wid], [ELUC_val, ELUC_val], '-', color=col_ELUC, linewidth=2, zorder=29.5)
    ax.plot([x1 - 0.07, x4 + 3/4 * 0.07], [ELUC_val, ELUC_val], '--', color='gray', linewidth=1, zorder=29)
    ax.plot([x2 - 0.07, x5 + 3/4 * 0.07], [SLAND_for_bot, SLAND_for_bot], '--', color='gray', linewidth=1, alpha=alpha_CAN, zorder=20)
    ax.plot([x3 - 3/5 * wid, x3 +  3/5 * wid], [NGHGI, NGHGI], '-', color='k', linewidth=2, zorder=29)
    ax.plot([x3 - 0.07, x5 + 3/4 * 0.07], [NGHGI, NGHGI], '-', color='k', linewidth=1, zorder=29)
    
    #Plot arrow for ELUC
    if np.abs(ELUC_val)<44/12/1000*60:
        shrELUC = 0.15
        l_head = 44/12/1000 * 15
    else:
        shrELUC = 0.2
        l_head = 44/12/1000 * 20

    y_arr1 = shrELUC * ELUC_val
    y_arr2 = ELUC_val - 2 * shrELUC * ELUC_val - l_head * np.sign(ELUC_val)
    if np.abs(ELUC_val) > 0.030:
        ax.arrow(x1, y_arr1 , 0.0, y_arr2, width=0.03, fc="white", ec="none", head_width=0.1, head_length=l_head, alpha=0.8, zorder=50)
    
    #Plot arrow for SLAND
    shrSLAND = 0.2
    l_head = 44/12/1000 * 20
    y_arr1 = ELUC_val + shrSLAND * SLAND_for_med
    y_arr2 = SLAND_for_med - 2 * shrSLAND * SLAND_for_med - l_head * np.sign(SLAND_for_med)
    if np.abs(SLAND_for_top) > 0.030:
        ax.arrow(x2, y_arr1, 0.0, y_arr2, width=0.03, fc="white", ec="none", head_width=0.1, head_length=l_head, alpha=0.8, zorder=50)
    
    #Plot arrow for NGHGI
    y_arr1 = shrSLAND * NGHGI
    y_arr2 = NGHGI - 2 * shrSLAND * NGHGI - l_head * np.sign(NGHGI)
    if np.abs(NGHGI) > 0.030:
        ax.arrow(x3, y_arr1, 0.0, y_arr2, width=0.03, fc="white", ec="none", head_width=0.1, head_length=l_head, alpha=0.8, zorder=50)

    #Make stripes in background
    if np.mod(i, 2)==0:
        ax.axvspan(i, i + 1, facecolor='black', alpha=0.05)
        
    #Define country names
    if country=='USA':        ctr_name = country
    elif country=='RUS':      ctr_name = 'Russia'
    elif country=='COD':      ctr_name = 'DR Congo'
    elif country=='TZA':      ctr_name = 'Tanzania'
    elif country=='CAF':      ctr_name = 'Cent. Afr. Rep.'
    elif country=='EU27_UK':  ctr_name = 'EU27&UK'
    else:                     ctr_name = data_cntrs.loc[country].item()
    ctrs.append(ctr_name)
    
    gap_before = gap_before + np.abs(NGHGI-ELUC_val)
    gap_after  = gap_after + np.abs(resid_val_top)

#Set x-ticks and x-tick labels
ax.set_xticks(np.arange(0.5, len(ctrs)))
ax.set_xticklabels(ctrs, rotation=0, ha='center', fontsize=14)

#Plot zero line
ax.plot([-10, len(ctrs) + 5], [0, 0], '-', color='lightgray', zorder=0)

#Set limits and y-label of second axis
ax.set_xlim([0, len(ctrs)])
if  include_SLANDtot==1:  ax.set_ylim([-0.460, 0.360])
else:                     ax.set_ylim([-1.300, 1.300])
ax.set_ylabel('CO$_2$ flux / Pg CO$_2$ yr$^{-1}$', fontsize=20, color='k', labelpad=15)    
ax.tick_params(labelsize=15)
ax.tick_params(axis='x', which='major', pad=10)
ax.tick_params(axis="x", which="both", bottom= False, top=False)

#Legend
leg1 = ax.legend([p1[0], p2[0], p4[0]], ['Anthropogenic LULUCF flux from models', 'Natural land flux in managed forests from models', 'LULUCF flux from country reports'], fontsize=fs_leg, frameon = False, loc=9, ncol=1, bbox_to_anchor=(0.275, -0.2))
ax.add_artist(leg1)
ax.legend([p5[0], p6[0]], ['Gap before including natural land flux', 'Gap after including natural land flux'], fontsize=fs_leg, frameon = False, loc=9, ncol=1, bbox_to_anchor=(0.79, -0.2))

#Save figure
fig.savefig(dir_fig + 'Fig2_NGHGI_composition_' + time_selection + '_ForMask' + forest_mask_year + '.png', dpi=300, bbox_inches='tight')
fig.savefig(dir_fig + 'Fig2_NGHGI_composition_' + time_selection + '_ForMask' + forest_mask_year + '.pdf', bbox_inches='tight')
